In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from env import host, user, password

# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

import os
os.path.isfile('telco_df.csv')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

import acquire

In [2]:
def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

# Exercises I - Required
- Let's review the steps we take at the beginning of each new module.
- #1. Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.
- #2. Clone this repository within your local codeup-data-science directory.
- #3. Create a .gitignore and make sure your list of 'files to ignore' includes your env.py file.
- #4. Ceate a README.md file that outlines the contents and purpose of your repository.
- #5. Add, commit, and push these two files.
- #6. Now you can add your env.py file to this repository to access the Codeup database server.
- #7. For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.
- #8. As always, add, commit, and push your work often.


# Exercises II - Required
- Let's set up an example scenario as perspective for our regression exercises using the Telco dataset.

- As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. You need to do this within an average of $5.00 per customer.

- In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Telco data from the telco_churn database in the Codeup database server.


In [3]:
#1. Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract.

In [4]:
#acquire telco dataframe
telco_df= acquire.get_telco_data()

In [5]:
telco_df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,1,1,1,4581-LNWUM,Female,0,No,No,13,No,...,No,Yes,Yes,No,49.15,649.4,No,Month-to-month,DSL,Electronic check
1,1,1,1,4580-TMHJU,Female,0,Yes,Yes,43,Yes,...,No,Yes,No,No,56.15,2499.3,Yes,Month-to-month,DSL,Electronic check
2,1,1,1,4570-QHXHL,Female,0,No,No,9,Yes,...,No,No,No,Yes,43.75,405.7,No,Month-to-month,DSL,Electronic check
3,1,1,1,4566-QVRRW,Female,0,Yes,No,11,Yes,...,No,No,No,Yes,44.05,483.7,Yes,Month-to-month,DSL,Electronic check
4,1,1,1,4537-DKTAL,Female,0,No,No,2,Yes,...,No,No,No,No,45.55,84.4,No,Month-to-month,DSL,Electronic check


In [6]:
telco_df['contract_type_id'].value_counts() # Look at unique values in contract_type_id column

1    3875
3    1695
2    1473
Name: contract_type_id, dtype: int64

In [7]:
#creating sql query to filter columns to customer_id, monthly_charges, tenure, total_charges, and contract_type_id
two_year_query= " Select customer_id, monthly_charges, tenure, total_charges FROM customers  JOIN contract_types USING(contract_type_id)  JOIN internet_service_types USING(internet_service_type_id) JOIN payment_types USING(payment_type_id) WHERE `contract_type_id` = '3';"

In [8]:
#using pandas to read sql query above 

telco_two_year_df= pd.read_sql(two_year_query, get_connection('telco_churn'))

In [9]:
telco_two_year_df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0016-QLJIS,90.45,65,5957.9
1,0017-DINOC,45.20,54,2460.55
2,0019-GFNTW,45.05,56,2560.1
3,0056-EPFBG,39.40,20,825.4
4,0078-XZMHT,85.15,72,6316.2


In [10]:
#2. Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. 
# You may handle the missing values however you feel is appropriate and meaningful; 
# remember to document your process and decisions using markdown and code commenting where helpful.

In [11]:
telco_two_year_df.head()#Looking at first 5 values

,customer_id,monthly_charges,tenure,total_charges
0,0016-QLJIS,90.45,65,5957.9
1,0017-DINOC,45.20,54,2460.55
2,0019-GFNTW,45.05,56,2560.1
3,0056-EPFBG,39.40,20,825.4
4,0078-XZMHT,85.15,72,6316.2


In [12]:

telco_two_year_df.shape # Looking at shape: 1695 rows and 4 columns

(1695, 4)

In [13]:
telco_two_year_df.describe().T

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


**From the list of datatypes shown below we need to change total_charges from an object to a numeric datatype.**
- total_charges has odd values causing it to be a object instead of a numeric datatype

In [14]:
telco_two_year_df.info() #Looking at datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


**Below we can see there are no null values present in our current dataframe.**

In [15]:
telco_two_year_df.isnull().sum() # check to see if there are any null values

customer_id        0
monthly_charges    0
tenure             0
total_charges      0
dtype: int64

In [16]:
# Check for any Null values in each column of our DataFrame.

telco_two_year_df.isnull().any()

customer_id        False
monthly_charges    False
tenure             False
total_charges      False
dtype: bool

**We can see below there are 10 columns in the total charges column that are empty values**

In [17]:
telco_two_year_df[telco_two_year_df['total_charges'] == ' '] # Looking at which columns in total charges that is empty

,customer_id,monthly_charges,tenure,total_charges
85,1371-DWPAZ,56.05,0,
156,2775-SEFEE,61.90,0,
236,4075-WKNIU,73.35,0,
255,4472-LVYGI,52.55,0,
339,5709-LVOEQ,80.85,0,
1221,2520-SGTTA,20.00,0,
1257,3115-CZMZD,20.25,0,
1267,3213-VVOLG,25.35,0,
1338,4367-NUYAO,25.75,0,
1547,7644-OMVMY,19.85,0,


**How do we handle the empty values?**
- Need to add zero to be able to convert to a numeric datatype b/c if you try to change an empty column to numeric it will produce an error.

In [18]:
#Add zero to columns to convert to float
telco_two_year_df['total_charge00s'] = telco_two_year_df['total_charges'] + '0'

In [19]:
#make total charges into datatype float
telco_two_year_df['total_charges'] = telco_two_year_df['total_charges'].astype('float')

In [20]:
telco_two_year_df.info() # making sure total_charges was changed to a float datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


In [21]:
#3. End with a wrangle.py file that contains the necessary functions to automate your process from 
# acquiring the data to returning a cleaned dataframe with no missing values. 
# Name your final function wrangle_telco.

**Making wrangle_telco function and making sure it works**

In [22]:
def new_telco_data():
    '''
    This function reads data from the Codeup db into a df.
    '''
    telco_sql = "SELECT * \
                 FROM customers \
                 JOIN contract_types USING(contract_type_id) \
                 JOIN internet_service_types USING(internet_service_type_id)\
                 JOIN payment_types USING(payment_type_id);"    
    return pd.read_sql(telco_sql, get_connection('telco_churn'))

In [23]:
def get_telco_data(cached=False):
    '''
    This function reads in telco churn data from Codeup database and writes data to
    a csv file if cached == False or if cached == True reads in telco df from
    a csv file, returns df.
    '''
    if cached == False or os.path.isfile('telco_df.csv') == False:
        
        # Read fresh data from db into a DataFrame.
        df = new_telco_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('telco_df.csv')
        
    else:
        
        # If csv file exists or cached == True, read in data from csv.
        df = pd.read_csv('telco_df.csv', index_col=0)
        
    return df


In [24]:
def telco_two_year(df):
    query= "Select customer_id, monthly_charges, tenure, total_charges FROM customers JOIN contract_types USING(contract_type_id) JOIN internet_service_types USING(internet_service_type_id)JOIN payment_types USING(payment_type_id) WHERE `contract_type_id` = '3';"
    df= pd.read_sql(query, get_connection('telco_churn'))
    return df

In [25]:
def clean_data(df):
    '''
    This function take in the telco dataframe created and clean the total charges column
    by add a 0 to all empty columns and change the column to a float datatype.
    Returns new dataframe with total charges column cleaned.
    '''
    #Add zero to columns to convert to float
    df['total_charges'] = df['total_charges'] + '0'
    #make total charges into datatype float
    df['total_charges'] = df['total_charges'].astype('float')
    return df
    

In [26]:
def telco_split(df):
    '''
    This function take in the telco data acquired by get_telco_data,
    performs a split.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123)
    return train, validate, test


**Created wrangle_telco function**

In [27]:
def wrangle_telco(df):
    df = clean_data(get_telco_data())
    return telco_split (df)

In [28]:
#acquire telco dataframe
telco_copy= acquire.get_telco_data()

In [29]:
wrangle= wrangle_telco(telco_df)

In [31]:
wrangle

(      payment_type_id  internet_service_type_id  contract_type_id customer_id  \
 4604                3                         2                 1  5564-NEMQO   
 5566                4                         3                 3  0825-CPPQH   
 6204                2                         3                 2  1561-BWHIN   
 5837                3                         3                 3  4979-HPRFL   
 1276                1                         1                 1  0749-IRGQE   
 ...               ...                       ...               ...         ...   
 1794                2                         1                 1  4827-USJHP   
 817                 2                         1                 2  3417-TSCIC   
 901                 4                         1                 2  4825-XJGDM   
 1987                1                         1                 1  6410-LEFEN   
 6958                3                         3                 1  8204-TIFGJ   
 
       gender 